<a href="https://colab.research.google.com/github/Stefano97/AN2DL_Project2/blob/master/transferLearningAttempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

SEED = 1234
tf.random.set_seed(SEED) 

In [ ]:
!git clone https://github.com/Stefano97/AN2DL_Project2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp -R "/content/drive/My Drive/ann-and-dl-image-segmentation.zip" "/content/" 

In [ ]:
!unzip ./*.zip

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data_augumentation = True

if data_augumentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                              width_shift_range=10,
                                              height_shift_range=10,
                                              zoom_range=0.3,
                                              horizontal_flip=True,
                                              vertical_flip=True,
                                              fill_mode='constant', 
                                              cval=0,
                                              rescale=1./255,
                                              validation_split=0.3)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                              width_shift_range=10,
                                              height_shift_range=10,
                                              zoom_range=0.3,
                                              horizontal_flip=True,
                                              vertical_flip=True,
                                              fill_mode='constant', 
                                              cval=0,
                                              rescale = 1./255, 
                                              validation_split=0.3)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255,validation_split=0.3)
    train_mask_data_gen = ImageDataGenerator(rescale = 1./255,validation_split=0.3)


## Import dataset in the generators

In [ ]:
import os

cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'Segmentation_Dataset')

# Batch size
bs = 4

# img shape
img_h = 256
img_w = 256

#num_classes=21

# Training
# Two different generators for images and masks
# ATTENTION: here the seed is important!! We have to give the same SEED to both the generator
# to apply the same transformations/shuffling to images and corresponding masks
training_dir = os.path.join(dataset_dir, 'training')
train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset="training")  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode="grayscale",
                                                         seed=SEED,
                                                         subset="training")
valid_img_gen = train_img_data_gen.flow_from_directory(os.path.join(training_dir, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED,
                                                       subset="validation")  
valid_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(training_dir, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode="grayscale",
                                                         seed=SEED,
                                                         subset="validation")

train_gen = zip(train_img_gen, train_mask_gen)
valid_gen = zip(valid_img_gen, valid_mask_gen)

In [ ]:
# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))

def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.int32)
    return x_, y_

train_dataset = train_dataset.map(prepare_target)

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, img_h, img_w, 1]))
valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()


## Create JSON file

In [ ]:
import json
import re

training = []
for imgName in train_img_gen.filenames:
    matchObj = re.search(r'[^\\\\/]+$',imgName,re.M)    
    file_name = matchObj.group()
    training.append(file_name)
    
validation = []
for imgName in valid_img_gen.filenames:
    matchObj = re.search(r'[^\\\\/]+$',imgName,re.M)    
    file_name = matchObj.group()
    validation.append(file_name)

# dictionary with the format shown in the Evaluation tab
dataset_split = {'training': training , 'validation': validation}   
with open('dataset_split.json', 'w') as fp:
    json.dump(dataset_split, fp)




## Show images in the generator

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook


fig,ax = plt.subplots(1,2)
fig.show()

iterator = iter(train_gen)

for _ in range(1000):
    img, target = next(iterator)
    img = img[0]   # First element
    img = img * 255  # denormalize
    target = target[0]
    
    target = target.squeeze(-1)
    
    
    # Assign colors (just for visualization)
    target_img = np.zeros([256, 256, 3])

    target_img[np.where(target == 0.0)] = [0, 0, 0]
    target_img[np.where(target == 1.0)] = [255, 255, 255]


    ax[0].imshow(np.uint8(img))
    ax[1].imshow(np.uint8(target_img))


    fig.canvas.draw()
    time.sleep(1)

## ENCODER AND DECODER - BUILD A MODEL 
### (already seen in the exercise classes)


In [ ]:
#first attempt

def create_model(depth, start_f, num_classes, dynamic_input_shape):

    model = tf.keras.Sequential()
    
    # Encoder
    # -------
    for i in range(depth):
        
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, 3]
            else:
                input_shape = [img_h, img_w, 3]
        else:
            input_shape=[None]
        
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                         kernel_size=(3, 3),
                                         kernel_initializer='he_normal',
                                         strides=(1, 1),
                                         padding='same',
                                         input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.Conv2DTranspose(start_f // 2,(3,3),strides=(2,2),padding='same'))
        model.add(tf.keras.layers.concatenate())
        model.add(tf.keras.layers.Conv2D(filters=start_f // 2,
                                         kernel_size=(3, 3),
                                         strides=(1, 1),
                                         padding='same'))

        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(tf.keras.layers.Conv2D(filters=num_classes,
                                     kernel_size=(1, 1),
                                     strides=(1, 1),
                                     padding='same',
                                     activation='sigmoid'))
    
    return model

model = create_model(depth=4, 
                     start_f=4, 
                     num_classes=1, 
                     dynamic_input_shape=False)

# Visualize created model as a table
model.summary()
model.weights


## test_IoU = 0.032% ---> very bad

## PREPARE THE MODEL FOR TRAINING


In [ ]:

# Loss
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True) 

lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=[my_IoU])

In [ ]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'segmentation_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms


# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='my_IoU', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_img_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_img_gen), 
          callbacks=callbacks)

In [ ]:
model.load_weights('./segmentation_experiments/CNN_Dec14_18-36-15/ckpts/cp_10.ckpt')

## TEST WITH KAGGLE CREATING FILE CSV

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./output/'):
    if not os.path.exists(results_dir):
      os.makedirs(results_dir)

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

      f.write('ImageId,EncodedPixels,Width,Height\n')

      for key, value in results.items():
          f.write(key + ',' + value + ',' + '256' + ',' + '256' + '\n')

In [ ]:
import os
from PIL import Image
import numpy as np

def rle_encode(img):
      # Flatten column-wise
      pixels = img.T.flatten()
      pixels = np.concatenate([[0], pixels, [0]])
      runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
      runs[1::2] -= runs[::2]
      return ' '.join(str(x) for x in runs)

image_filenames = next(os.walk('./Segmentation_Dataset/test/images/img/'))[2]                          
results = {}

for image_name in image_filenames:
    img = Image.open('./Segmentation_Dataset/test/images/img/'+image_name).convert('RGB')
    img_array = np.array(img)
    #img_array = img_array*1./255 #normalization
    img_array = np.expand_dims(img_array,0) #needed for fixed dim of input in the model
    output = model.predict(img_array)
    output = tf.cast(output > 0.5, tf.int32) 
    output = output.numpy()
    results[image_name[:-4]] = rle_encode(output)
    
create_csv(results)
